In [11]:
import os
import numpy as np
from InputDataProcessor import InputDataProcessor 
from GenericModel import GenericModel
from distributeFunctions import distribute_602020
from dataFilters import csv_weather_data_prediction_filter
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError
from tensorflow.keras.optimizers import Adam
from ModelConfig import ModelConfig

In [12]:
idp=InputDataProcessor(bucket_name='aiml_harveen', use_case='weather_elbasan/regression', file_name='elbasan2021.csv')

In [13]:
idp.prepareCsvModelInputData(csv_weather_data_prediction_filter,distribute_602020)

In [14]:
os.environ["AIP_TRAINING_DATA_URI"] = "gs://aiml_harveen/weather_elbasan/regression/training_data/elbasan2021.csv"
os.environ["AIP_VALIDATION_DATA_URI"] = "gs://aiml_harveen/weather_elbasan/regression/dev_data/elbasan2021.csv"
os.environ["AIP_TEST_DATA_URI"] = "gs://aiml_harveen/weather_elbasan/regression/test_data/elbasan2021.csv"
os.environ["AIP_MODEL_DIR"] = "gs://aiml_harveen/weather_elbasan/regression/weather_model"

In [15]:
modelConfigList=[
    ModelConfig(
        model_type="regression",
        layers=["relu","relu","linear"],
        neurons=[16,8,1],
        epochs=20,
        loss=MeanSquaredError(),
        optimizer=Adam(learning_rate=0.01),
        regularizers=[0.001,0.001,0.001]),
    
    ModelConfig(
        model_type="regression",
        layers=["relu","relu","linear"],
        neurons=[24,16,1],
        epochs=20,
        loss=MeanSquaredError(),
        optimizer=Adam(learning_rate=0.01),
        regularizers=[0.001,0.001,0.001]),

    ModelConfig(
        model_type="regression",
        layers=["relu","relu","linear"],
        neurons=[32,16,1],
        epochs=20,
        loss=MeanSquaredError(),
        optimizer=Adam(learning_rate=0.01),
        regularizers=[0.001,0.001,0.001]),
]

In [16]:
trainErrAry=np.zeros([len(modelConfigList)])
validationErrAry=np.zeros([len(modelConfigList)])



In [17]:
for idx,modelConfig in enumerate(modelConfigList):
    gm=GenericModel(modelConfig)
    gm.train()
    trainErrAry[idx] = gm.train_err
    gm.validate()
    validationErrAry[idx] = gm.validation_err

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 16)                112       
                                                                 
 dense_13 (Dense)            (None, 8)                 136       
                                                                 
 dense_14 (Dense)            (None, 1)                 9         
                                                                 
Total params: 257
Trainable params: 257
Non-trainable params: 0
_________________________________________________________________
3/3 [==============================] - 0s 2ms/step
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 24)                168       
                                                 

In [18]:
trainErrAry

array([15.72109295, 13.29980612, 11.14596424])

In [19]:
validationErrAry

array([19.87382403, 20.44225402, 32.56417939])

In [9]:
mseAry

array([[14.1906579 , 15.80274403],
       [12.09241874, 21.42322218],
       [11.21213009, 17.96227199]])

In [22]:
#predict(avg_temp=13.9,min_temp=10,max_temp=17.4,wind_dir=73,wind_speed=5.9,pressure=1010.3)